### Importing Packages

In [1]:
import sys
sys.path.insert(0, 'C:/Users/enioh/Documents/Github/MMA-ML-Model')

from os.path import abspath, dirname, join

import pandas as pd
import numpy as np

### Importing Fully Processed Data

In [2]:
df = pd.read_csv(abspath(join(dirname(abspath('')), 'data/', 'processed/', 'extracted_stats.csv')))

## MODEL 1: XGBoost 

In [ ]:
from src.models.XGBoost_Model import make_model
from src.models.train_model import train_model

clf = make_model()
clf, data = train_model(clf)

In [ ]:
X, y = data
y[y == 'W'] = 1
y[y != 1] = 0
y = y.to_numpy(dtype=int)

### Not that good, lets search for hyperparameters

In [5]:
from src.models.Optuna import OptunaTuning

tuner = OptunaTuning(X,y)
tuner.run()

In [73]:
from src.features.feature_selection import get_corr_features

df['date'] = pd.to_datetime(df['date']) # make sure logic can handle datetime for time series data
df.drop(df.loc[df['result'] == 'D'].index, inplace=True) # Drop ties first 
drop_cols = df.loc[:, :'referee'].columns # columns that will be dropped as features (strings, formats, etc.)
drop_cols = drop_cols.insert(0, df.columns[df.isna().mean() > 0.9].to_list()) # drop columns that contain more than % nans
post_comp_cols = df.loc[:, ~df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)].columns.to_list() # All columns that do not have the precomp_ id or vs_opp id
drop_cols = drop_cols.insert(0, post_comp_cols)
features = df.drop(columns=drop_cols, axis=1)
drop_features_cols = get_corr_features(features, thresh=0.95)
drop_cols = drop_cols.insert(0, drop_features_cols)

multi_df = df.set_index(['fighter_url', df.groupby('fighter_url').cumcount()])


C:\Users\enioh\AppData\Local\Temp\ipykernel_8644\1169957758.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  post_comp_cols = df.loc[:, ~df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)].columns.to_list() # All columns that do not have the precomp_ id or vs_opp id


In [68]:
#drop_cols = drop_cols.drop_duplicates().drop('fighter_url')
multi_df = multi_df.drop(columns=drop_cols, axis=1)

In [118]:
z = multi_df.isna().mean(axis=1).groupby('fighter_url').mean() < 0.5

In [35]:
idx = pd.IndexSlice

In [119]:
z = multi_df.loc[(z[z].index, slice(None)), ['result']]

In [122]:
z

result
fighter_url                                                
http://ufcstats.com/fighter-details/0052de90691... 0      L
                                                   1      L
                                                   2      W
                                                   3      W
                                                   4      W
...                                                     ...
http://ufcstats.com/fighter-details/ffe9703408f... 0      W
                                                   1      L
                                                   2      W
                                                   3      W
                                                   4      L

[10535 rows x 1 columns]

In [124]:
zz = z.to_xarray().to_array().to_numpy()

In [129]:
np.transpose(zz, (1, 0, 2))[0]

array([['L', 'L', 'W', 'W', 'W', 'W', 'L', nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan]], dtype=object)

In [60]:
multi_df

date  \
fighter_url                                                        
http://ufcstats.com/fighter-details/9e8f6c728eb... 0  2023-07-29   
                                                   1  2023-03-18   
                                                   2  2022-05-07   
                                                   3  2021-11-06   
                                                   4  2020-10-24   
...                                                          ...   
http://ufcstats.com/fighter-details/0e041d43a47... 10 2001-05-04   
                                                   11 2000-12-16   
http://ufcstats.com/fighter-details/56b8b24f65f... 0  2005-02-05   
                                                   1  2004-08-21   
                                                   2  2004-04-02   

                                                                                               fight_url  \
fighter_url                                                                                                
http://ufcstats.com/fighter-details/9e8f6c728eb... 0   http://ufcstats.com/fight-details/c27e33ee8ef5...   
                                                   1   http://ufcstats.com/fight-details/90a83875e1d1...   
                                                   2   http://ufcstats.com/fight-details/7d4e49d8a667...   
                                                   3   http://ufcstats.com/fight-details/999a0fad4c82...   
                                                   4   http://ufcstats.com/fight-details/59eea37af7ef...   
...                                                                                                  ...   
http://ufcstats.com/fighter-details/0e041d43a47... 10  http://ufcstats.com/fight-details/ef99fb08c6b0...   
                                                   11  http://ufcstats.com/fight-details/c39728802a84...   
http://ufcstats.com/fighter-details/56b8b24f65f... 0   http://ufcstats.com/fight-details/48189cad0c11...   
                                                   1   http://ufcstats.com/fight-details/c74eef9b3376...   
                                                   2   http://ufcstats.com/fight-details/3b8bfe14b0c2...   

                                                                                               event_url  \
fighter_url                                                                                                
http://ufcstats.com/fighter-details/9e8f6c728eb... 0   http://ufcstats.com/event-details/ccd58ff71e26...   
                                                   1   http://ufcstats.com/event-details/e4bb7e483c4a...   
                                                   2   http://ufcstats.com/event-details/253d3f9e97ca...   
                                                   3   http://ufcstats.com/event-details/48e093ea1f43...   
                                                   4   http://ufcstats.com/event-details/c3c38c86f5ab...   
...                                                                                                  ...   
http://ufcstats.com/fighter-details/0e041d43a47... 10  http://ufcstats.com/event-details/20bd6c3e03c4...   
                                                   11  http://ufcstats.com/event-details/08ae5cd9aef7...   
http://ufcstats.com/fighter-details/56b8b24f65f... 0   http://ufcstats.com/event-details/85d905f7c4f5...   
                                                   1   http://ufcstats.com/event-details/b4bc2e3353a7...   
                                                   2   http://ufcstats.com/event-details/e8fb8e53bc2e...   

                                                      result         fighter  \
fighter_url                                                                    
http://ufcstats.com/fighter-details/9e8f6c728eb... 0       W  Justin Gaethje   
                                                   1       W  Justin Gaethje   
                                             